In [3]:
from keras.models import *
from keras.layers import *
from keras import optimizers
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import cv2
import os
import numpy as np
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math

In [4]:
import numpy as np
import os
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
import tensorflow as tf

def jaccard_loss(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def castF(x):
    return K.cast(x, K.floatx())

def castB(x):
    return K.cast(x, bool)

def iou_loss_core(true,pred):  #this can be used as a loss if you make it negative
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)

    return (K.sum(intersection, axis=-1) + K.epsilon()) / (K.sum(union, axis=-1) + K.epsilon())

def IOU(true, pred): #any shape can go - can't be a loss function

    tresholds = [0.5 + (i*.05)  for i in range(10)]

    #flattened images (batch, pixels)
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = castF(K.greater(pred, 0.5))

    #total white pixels - (batch,)
    trueSum = K.sum(true, axis=-1)
    predSum = K.sum(pred, axis=-1)

    #has mask or not per image - (batch,)
    true1 = castF(K.greater(trueSum, 1))    
    pred1 = castF(K.greater(predSum, 1))

    #to get images that have mask in both true and pred
    truePositiveMask = castB(true1 * pred1)

    #separating only the possible true positives to check iou
    testTrue = tf.boolean_mask(true, truePositiveMask)
    testPred = tf.boolean_mask(pred, truePositiveMask)

    #getting iou and threshold comparisons
    iou = iou_loss_core(testTrue,testPred) 
    truePositives = [castF(K.greater(iou, tres)) for tres in tresholds]

    #mean of thressholds for true positives and total sum
    truePositives = K.mean(K.stack(truePositives, axis=-1), axis=-1)
    truePositives = K.sum(truePositives)

    #to get images that don't have mask in both true and pred
    trueNegatives = (1-true1) * (1 - pred1) # = 1 -true1 - pred1 + true1*pred1
    trueNegatives = K.sum(trueNegatives) 

    return (truePositives + trueNegatives) / castF(K.shape(true)[0])
  
def _unet(pretrained_weights=None, input_size=(64, 224, 3)):
    inputs = Input(input_size, name="input_tensor")
    conv1 = Conv2D(32, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(conv2)

    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(drop2)
    
    
# -------------------------------------------------------------
    conv3 = Conv2D(128, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(conv3)
    drop3 = Dropout(0.5)(conv3)
# -------------------------------------------------------------
    up4 = Conv2D(64, 2, activation='relu', padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop3))
    merge4 = concatenate([drop2, up4], axis=3)
    conv4 = Conv2D(64, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(merge4)
    conv4 = Conv2D(64, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(conv4)

    up5 = Conv2D(32, 2, activation='relu', padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv4))
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(32, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(32, 3, activation='relu', padding='same',
                   kernel_initializer='he_normal')(conv5)
    
    conv6 = Conv2D(1, 1, activation='sigmoid', name="output_tensor")(conv5)

    model = Model(inputs, conv6)

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

def unet(pretrained_weights=None, input_size=(256, 336, 3)):
    model = _unet(pretrained_weights, input_size)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy', metrics=[IOU])
    return model


In [10]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications import MobileNetV2, ResNet50
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Sequential, Model
from tensorflow import keras
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
import logging
import os
import time
import numpy as np
import math
import cv2
import pickle



def _list_dir_abspath(dirpath):
    assert os.path.isdir(dirpath), "No such a directory"
    return [os.path.join(dirpath, elepath) for elepath in os.listdir(dirpath)]

def prepare_all_data(folder):
    list_subdataset = _list_dir_abspath(folder)
    inputs = list()
    outputs = list()
    for subdataset in list_subdataset:
        inputs_path = os.path.join(subdataset, "data_1")
        outputs_path = os.path.join(subdataset, "data_1_segmented")
        for filename in os.listdir(inputs_path):
            inp = os.path.join(inputs_path, filename)
            oup = os.path.join(outputs_path, filename)
            if not (filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg")):
                continue
            if os.path.isfile(inp) and os.path.isfile(oup):
                inputs.append(inp)
                outputs.append(oup)
    return np.stack(inputs), np.stack(outputs)


def find_images(path):
    path = os.path.abspath(path)
    assert os.path.isdir(path), ("No such a directory")
    cwd = os.getcwd()
    os.chdir(path)
    lst_imgs = []
    for fil in os.listdir():
        if not (fil.endswith(".png") or fil.endswith(".jpg") or fil.endswith(".jpeg")):
            continue
        lst_imgs.append(os.path.join(path, fil))
    os.chdir(cwd)
    return lst_imgs


def create_dataset(folder):
    return prepare_all_data(folder)


class DataSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size, datagen=None):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.datagen = datagen

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def preprecess_output(self, img_path, transform_parameters):
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, (256,256))
        if self.datagen is not None:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            img = self.datagen.apply_transform(img, transform_parameters)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(img, axis=-1)
        img = img/255.0
        return np.round(img).astype(np.uint8)

    def preprocess_input(self, img_path, transform_parameters):
        img = cv2.imread(img_path)
        img = img[:,:,:3]
        img = cv2.resize(img, (256,256))
        if self.datagen is not None:
            img = self.datagen.apply_transform(img, transform_parameters)
        img = img/255.0
        return img

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        if self.datagen is not None:
            transform_parameters = self.datagen.get_random_transform((256,256,3))
        else:
            transform_parameters = None
        inputs = {
            "input_tensor": np.array([self.preprocess_input(file_name, transform_parameters) for file_name in batch_x])
        }
        outputs = {
            "output_tensor": np.array([self.preprecess_output(file_name, transform_parameters) for file_name in batch_y])
        }
        return inputs, outputs


TRAIN_FOLDER = "test"
print("Reading data")
st = time.time()
x_dataset, y_dataset = create_dataset(TRAIN_FOLDER)
x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, test_size=0.2, random_state=42)
print("Collected {} images".format(len(x_train)))
# exit(0)
BATCH_SIZE = 16
TRAIN_SIZE = x_train.shape[0]
print("Reading data done in {}s".format(time.time() - st))
print(x_train.shape)
print(y_train.shape)
# datagen = ImageDataGenerator(brightness_range=[0.7, 1.0], width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2)
datagen = ImageDataGenerator(height_shift_range=0.1, width_shift_range=0.1, zoom_range=0.1)
train_gen = DataSequence(x_train, y_train, BATCH_SIZE, datagen)
test_gen = DataSequence(x_test, y_test, BATCH_SIZE)
my_model = unet(input_size=(256, 256, 3))
# my_model.compile(optimizer)
mc = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, period=1)
logs = tf.keras.callbacks.TensorBoard(log_dir="logs", histogram_freq=1)
csv_logger = tf.keras.callbacks.CSVLogger("history.csv", append=True)
my_model.fit_generator(train_gen, steps_per_epoch=TRAIN_SIZE//BATCH_SIZE, validation_data=test_gen, 
                        callbacks=[csv_logger, logs, mc], epochs=100, max_queue_size=32, 
                        workers=12, initial_epoch=0)


Reading data


FileNotFoundError: [WinError 3] The system cannot find the path specified: 't\\data_1'